# San Francisco Housing Cost Analysis

In this assignment, you will perform fundamental analysis for the San Francisco housing market to allow potential real estate investors to choose rental investment properties.

In [1]:
# imports
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

## Load Data

In [2]:
# Read the census data into a Pandas DataFrame
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


- - - 

## Housing Units Per Year

In this section, you will calculate the number of housing units per year and visualize the results as a bar chart using the Pandas plot function.

**Hint:** Use the Pandas `groupby` function.

**Optional challenge:** Use the min, max, and std to scale the y limits of the chart.



In [ ]:
# Calculate the mean number of housing units per year (hint: use groupby) 
units_per_year = sfo_data.groupby('year').mean('housing_units')
units_per_year.drop(columns=['sale_price_sqr_foot','gross_rent'], inplace=True)
units_per_year


In [ ]:
# Save the dataframe as a csv file
units_per_year.to_csv('Data/units_per_year.csv')

In [ ]:
# Use the Pandas plot function to plot the average housing units per year.
# Note: You will need to manually adjust the y limit of the chart using the min and max values from above.


# Optional Challenge: Use the min, max, and std to scale the y limits of the chart


# Plotting
units_per_year.plot(y='housing_units', kind='bar', title='Average Housing Units Per Year')
plt.xlabel('Year')
plt.ylabel('Average Housing Units')

# Adjust y limits based on min, max, and std
plt.ylim(units_per_year['housing_units'].min() - units_per_year['housing_units'].std(), units_per_year['housing_units'].max() + units_per_year['housing_units'].std())

# Show plot
plt.show()






- - - 

## Average Housing Costs in San Francisco Per Year

In this section, you will calculate the average monthly rent and the average price per square foot for each year. An investor may wish to better understand the sales price of the rental property over time. For example, a customer will want to know if they should expect an increase or decrease in the property value over time so they can determine how long to hold the rental property.  Plot the results as two line charts.

**Optional challenge:** Plot each line chart in a different color.

In [ ]:
# Calculate the average sale price per square foot and average gross rent
costs = sfo_data.groupby('year').mean(['gross_rent','sale_price_sqr_foot'])
costs.drop(columns=['housing_units'],inplace=True)
costs = costs.round(2)
costs

In [ ]:
# Create two line charts, one to plot the average sale price per square foot and another for average montly rent
# Line chart for average sale price per square foot
costs.plot(y='sale_price_sqr_foot', kind='line', title='Cost Per Square Foot by Year')
plt.xlabel('Year')
plt.ylabel('Sale Price per Square Foot')
plt.show()

# Line chart for average montly rent
costs.plot(y='gross_rent', kind='line', title='Rent by Year')
plt.xlabel('Year')
plt.ylabel('Gross Rent')
plt.show()



- - - 

## Average Prices by Neighborhood

In this section, you'll create a function named average_price_by_neighborhood to analyze and visualize the housing market trends in a specific San Francisco neighborhood. First, it filters housing data for the chosen neighborhood. Then, it cleans the data, ensuring sale prices are numeric and removes any missing values. Next, it calculates the yearly average sale price per square foot. Finally, it generates a line plot displaying this trend over time. The function will use Plotly Express for visualization, emphasizing clarity with labeled axes and a descriptive title. Upon calling this function with a neighborhood's name, it will return the trend plot. The same steps will be followed to analyze average gross rent trends.

In [ ]:
def average_price_by_neighborhood(neighborhood):
    # Filter the data for the specified neighborhood
    df_prices = sfo_data[sfo_data['neighborhood'] == neighborhood]
    
    # Convert 'sale_price_sqr_foot' to a numeric type, ignoring errors to avoid conversion issues
    df_prices['sale_price_sqr_foot'] = pd.to_numeric(df_prices['sale_price_sqr_foot'], errors='coerce')
    
    # Drop rows with NaN values in 'sale_price_sqr_foot' after conversion
    df_prices = df_prices.dropna(subset=['sale_price_sqr_foot'])
    
    # Group by 'year' and calculate the mean of 'sale_price_sqr_foot'
    df_avg_price_per_year = df_prices.groupby('year')['sale_price_sqr_foot'].mean().reset_index()
    
    # Create and return the plot
    plt.figure(figsize=(10, 6))
    plt.plot(df_avg_price_per_year['year'], df_avg_price_per_year['sale_price_sqr_foot'], marker='o')
    plt.title(f'Average Sale Price per Square Foot in {neighborhood}')
    plt.xlabel('Year')
    plt.ylabel('Average Sale Price per Square Foot')
    plt.grid(True)
    plt.show()


In [ ]:
# Test your function by passing a neighborhood name.
average_price_by_neighborhood("Bayview")

In [ ]:
# Use plotly to create an interactive line chart of the average monthly rent.
def average_rent_by_neighborhood(neighborhood):
    df_rent = sfo_data[sfo_data['neighborhood'] == neighborhood]

    # Convert 'gross_rent' to a numeric type, ignoring errors to avoid conversion issues
    df_rent['gross_rent'] = pd.to_numeric(df_rent['gross_rent'], errors='coerce')

    # Drop rows with NaN values in 'gross_rent' after conversion
    df_rent = df_rent.dropna(subset=['gross_rent'])

    # Group by 'year' and calculate the mean of 'gross_rent'
    df_avg_rent_per_year = df_rent.groupby('year')['gross_rent'].mean().reset_index()

    # Create an interactive line plot using Plotly
    fig = px.line(df_avg_rent_per_year, x='year', y='gross_rent', title=f'Average Monthly Rent in {neighborhood}')
    fig.update_layout(xaxis_title='Year', yaxis_title='Average Monthly Rent')
    fig.show()


In [ ]:
average_rent_by_neighborhood("Bayview")

## The Top 10 Most Expensive Neighborhoods

In this section, you will Group by year and neighborhood and then create a new dataframe of the mean values to calculate the mean sale price per square foot for each neighborhood and then sort the values to obtain the top 10 most expensive neighborhoods on average. Plot the results as a bar chart.

In [ ]:
# Getting the data from the top 10 expensive neighborhoods to own
df_mean_price = sfo_data.groupby(['year', 'neighborhood'])['sale_price_sqr_foot'].mean().reset_index()

# Group by neighborhood and calculate the overall mean sale price per square foot
df_neighborhood_mean = df_mean_price.groupby('neighborhood')['sale_price_sqr_foot'].mean().reset_index()

# Sort the values to obtain the top 10 most expensive neighborhoods
df_top_10 = df_neighborhood_mean.sort_values(by='sale_price_sqr_foot', ascending=False).head(10)

df_top_10 = df_top_10.round(2)
df_top_10

# Filter the original mean price DataFrame to include only the top 10 neighborhoods
df_top_10_neighborhoods = df_top_10['neighborhood']
df_top_10_filtered = df_mean_price[df_mean_price['neighborhood'].isin(df_top_10_neighborhoods)]

df_top_10_filtered



In [ ]:
# Plotting the data from the top 10 expensive neighborhoods
fig = px.bar(df_top_10, x='neighborhood', y='sale_price_sqr_foot', 
                title='Top 10 Most Expensive Neighborhoods in San Francisco',
                labels={'sale_price_sqr_foot': 'Average Sale Price per Square Foot'},
                color='neighborhood')
fig.update_layout(xaxis_title='Neighborhood', yaxis_title='Average Sale Price per Square Foot')

# Show the plot
fig.show()

# Alternative: Over Time
fig = px.line(df_top_10_filtered, x='year', y='sale_price_sqr_foot', color='neighborhood',
              title='Top 10 Most Expensive Neighborhoods in San Francisco Over Time',
              labels={'sale_price_sqr_foot': 'Average Sale Price per Square Foot'})
fig.update_layout(xaxis_title='Year', yaxis_title='Average Sale Price per Square Foot')
fig.show()


- - - 

## Comparing cost to purchase versus rental income

In this section, you will define a function that takes a selected neighborhood as input, filters the data for that neighborhood, creates a bar chart using Plotly Express, and returns the chart as a result.

In [3]:
def most_expensive_neighborhoods_rent_sales(selected_neighborhood):
    # Create a new DataFrame called df_costs containing only the rows from the DataFrame "sfo_data"
    df_costs = sfo_data[sfo_data['neighborhood'] == selected_neighborhood]
    df_costs.reset_index(inplace=True)

    # Generate a plotly bar chart comparing the sale_price_sqr_foot and gross_rent columns
    df_costs = df_costs[['year', 'sale_price_sqr_foot', 'gross_rent']].melt(id_vars='year', var_name='Cost Type', value_name='Value')
    
    fig = px.bar(df_costs, x='year', y='Value', color='Cost Type', barmode='group',
                 title=f'Sale Price per Square Foot and Gross Rent in {selected_neighborhood}')
    
    fig.update_layout(xaxis_title='Year', yaxis_title='Cost')
    
    return fig

In [4]:
# testing the function
most_expensive_neighborhoods_rent_sales('Union Square District')

- - - 

## Neighborhood Map

In this section, you will read in neighborhoods location data and build an interactive map with the average house value per neighborhood. Use a `scatter_mapbox` from Plotly express to create the visualization. 

### Load Location Data

In [ ]:
# Load neighborhoods coordinates data
locations = pd.read_csv('Data/neighborhoods_coordinates.csv')
locations

### Data Preparation

You will need to join the location data with the mean values per neighborhood.

1. Calculate the mean values for each neighborhood.

2. Join the average values with the neighborhood locations.

In [ ]:
# Calculate the mean values for each neighborhood
df_neighborhood_mean.rename(columns={'neighborhood':'Neighborhood'}, inplace=True)
df_neighborhood_mean.dropna(inplace=True)
df_neighborhood_mean


In [ ]:
df_neighborhood_mean['Neighborhood'] = df_neighborhood_mean['Neighborhood'].str.strip()
locations['Neighborhood'] = locations['Neighborhood'].str.strip()

# Join the average values with the neighborhood locations
df_map_data = df_neighborhood_mean.join(locations.set_index('Neighborhood'), on='Neighborhood', how='outer')
df_map_data.nunique()
df_map_data

### Mapbox Visualization

Plot the average values per neighborhood using a Plotly express `scatter_mapbox` visualization.

In [ ]:
# Create a scatter mapbox to analyze neighborhood info

fig = px.scatter_mapbox(df_map_data, lat="Lat", lon="Lon", hover_name="Neighborhood",
                        hover_data=["sale_price_sqr_foot"], color="sale_price_sqr_foot", size="sale_price_sqr_foot",
                        color_continuous_scale=px.colors.diverging.RdYlGn[::-1], size_max=25, zoom=11,
                        title="Average Sale Price per Square Foot by Neighborhood")

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

- - -

## Cost Analysis - Optional Challenge

In this section, you will use Plotly express to create visualizations that investors can use to interactively filter and explore various factors related to the house value of the San Francisco's neighborhoods. 

### Create a DataFrame showing the most expensive neighborhoods in San Francisco by year

In [192]:
# Fetch the data from all expensive neighborhoods per year.
df_expensive_neighborhoods = sfo_data.groupby(by="Neighborhood").mean()
df_expensive_neighborhoods = df_expensive_neighborhoods.sort_values(
    by="sale_price_sqr_foot", ascending=False
).head(10)
df_expensive_neighborhoods = df_expensive_neighborhoods.reset_index()

df_expensive_neighborhoods

,Neighborhood,level_0,index,year,sale_price_sqr_foot,housing_units,gross_rent
0,Union Square District,199.333333,199.333333,2012.500000,903.993258,377427.50,2555.166667
1,Merced Heights,260.333333,260.333333,2014.000000,788.844818,380348.00,3414.000000
2,Miraloma Park,132.500000,132.500000,2011.750000,779.810842,375967.25,2155.250000
3,Pacific Heights,215.142857,215.142857,2013.000000,689.555817,378401.00,2817.285714
4,Westwood Park,343.500000,343.500000,2015.000000,687.087575,382295.00,3959.000000
5,Telegraph Hill,225.142857,225.142857,2013.000000,676.506578,378401.00,2817.285714
6,Presidio Heights,220.714286,220.714286,2013.000000,675.350212,378401.00,2817.285714
7,Cow Hollow,183.142857,183.142857,2013.000000,665.964042,378401.00,2817.285714
8,Potrero Hill,219.714286,219.714286,2013.000000,662.013613,378401.00,2817.285714
9,South Beach,146.000000,146.000000,2011.666667,650.124479,375805.00,2099.000000


### Create a parallel coordinates plot and parallel categories plot of most expensive neighborhoods in San Francisco per year


In [ ]:
sfo_data['Neighborhood'] = sfo_data['Neighborhood'].str.strip()
sfo_data.reset_index(inplace=True)
sfo_data.rename(columns={'neighborhood':'Neighborhood'},inplace=True)
all_data = sfo_data.join(locations.set_index('Neighborhood'), on='Neighborhood', how='outer')
all_data.dropna(inplace=True)


In [ ]:
all_data.drop(columns=['index','level_0'],inplace=True)


In [166]:
all_data

,year,Neighborhood,sale_price_sqr_foot,housing_units,gross_rent,Lat,Lon
0,2010,Alamo Square,291.182945,372560,1239,37.791012,-122.40210
59,2011,Alamo Square,272.527310,374507,1530,37.791012,-122.40210
118,2012,Alamo Square,183.099317,376454,2324,37.791012,-122.40210
176,2013,Alamo Square,387.794144,378401,2971,37.791012,-122.40210
233,2014,Alamo Square,484.443552,380348,3528,37.791012,-122.40210
...,...,...,...,...,...,...,...
117,2011,Yerba Buena,438.860545,374507,1530,37.792980,-122.39636
175,2012,Yerba Buena,491.814003,376454,2324,37.792980,-122.39636
232,2013,Yerba Buena,753.011413,378401,2971,37.792980,-122.39636
292,2014,Yerba Buena,479.923749,380348,3528,37.792980,-122.39636


In [193]:
# Parallel Categories Plot
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
all_data['Neighborhood_encoded'] = le.fit_transform(all_data['Neighborhood'])

fig = px.parallel_categories(df_expensive_neighborhoods, 
                             dimensions=['Neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
                             color='sale_price_sqr_foot',
                             color_continuous_scale=px.colors.diverging.Tealrose)

fig.show()


In [188]:
# Parallel Coordinates Plot
df_coor = all_data.groupby('Neighborhood').mean('sales_price_sqr_foot')
fig = px.parallel_coordinates(df_coor, color='sale_price_sqr_foot', 
                              dimensions=['sale_price_sqr_foot', 'housing_units', 'gross_rent'],
                              labels={
                                  'sale_price_sqr_foot': 'Sale Price per Square Foot',
                                  'housing_units': 'Housing Units',
                                  'gross_rent': 'Gross Rent'
                              },
                              color_continuous_scale=px.colors.sequential.Viridis)

# Show the plot
fig.show()

### Create a sunburst chart to conduct a costs analysis of most expensive neighborhoods in San Francisco per year

In [ ]:
# Sunburst Plot
df_top_7 = df_mean_price.groupby('year').apply(lambda x: x.nlargest(7, 'sale_price_sqr_foot')).reset_index(drop=True)

df_top_7 = df_top_7.sort_values(by='year')

# Creating the sunburst chart
fig = px.sunburst(df_top_7, 
                  path=['year', 'neighborhood'], 
                  values='sale_price_sqr_foot', 
                  title='Costs Analysis of Most Expensive Neighborhoods in San Francisco Per Year',
                  color='year',
                  color_continuous_scale=px.colors.sequential.Viridis[::-1])  # Reverse colormap for green low, red high

fig.update_layout(margin=dict(t=50, l=0, r=0, b=0))
fig.show()